In [ ]:
!pip install wandb

In [ ]:
from huggingface_hub import login
from google.colab import userdata

login(token=userdata.get('HF_TOKEN'))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
%%capture
import torch
major_version, minor_version = torch.cuda.get_device_capability()
# Must install separately since Colab has torch 2.2.1, which breaks packages
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    # Use this for new GPUs like Ampere, Hopper GPUs (RTX 30xx, RTX 40xx, A100, H100, L40)
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    # Use this for older GPUs (V100, Tesla T4, RTX 20xx)
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass

In [ ]:
from unsloth import FastLanguageModel

max_seq_length = 512 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Viet-Mistral/Vistral-7B-Chat",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

In [ ]:
tokenizer.pad_token = tokenizer.bos_token
model.config.pad_token_id = model.config.bos_token_id
tokenizer.padding_side = "right"

model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = True,
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [ ]:
import pandas as pd

dataset = pd.read_csv('/content/raw_data.csv')
dataset

In [ ]:
# Remove row if column 'content' is null
dataset = dataset.dropna(subset=['content'])
dataset

In [ ]:
EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(row):
    content = row['content']
    label = row['label']
    text = f'Xác định văn bản sau có phản động chống Đảng Cộng sản không. 0 là trong sạch, 1 là phản động: \nText: {content}\nCategory: {label}{EOS_TOKEN}'
    return text

dataset['formatted_text'] = dataset.apply(formatting_prompts_func, axis=1)

<h3> wandb log </h3>

In [ ]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
import random
random.seed(42)

from trl import SFTTrainer
from transformers import TrainingArguments

output_directory = "Vistral_antigov_detection"
max_seq_length = 512

random_indices_test = random.sample(range(len(dataset['test'])), 200) #
test_set = dataset['test'].select(random_indices_test)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset['train'],
    eval_dataset = test_set,
    dataset_text_field = 'text',
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 8,
        per_device_eval_batch_size = 8,
        gradient_accumulation_steps = 4,
        eval_accumulation_steps = 4,
        num_train_epochs = 1,
        warmup_steps = 1,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 2,
        optim = "adamw_8bit",
        report_to = ["tensorboard", "wandb"],
        weight_decay = 0.01,
        save_strategy = "epoch",
        evaluation_strategy='steps',
        lr_scheduler_type = 'linear',
        seed = 4269,
        output_dir = output_directory,
        push_to_hub = True,
    ),
)

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=2):   0%|          | 0/1000 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
4.625 GB of memory reserved.


In [ ]:
progress_callback = WandbPredictionProgressCallback(trainer, tokenizer, dataset['test'], num_samples=100)
# trainer.add_callback(progress_callback)

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 8 | Gradient Accumulation steps = 4
\        /    Total batch size = 32 | Total steps = 31
 "-____-"     Number of trainable parameters = 41,943,040


wandb: Currently logged in as: tohrumi (andynguyen). Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss
2,1.851100,1.563304
4,1.577000,1.300672
6,1.292100,1.243118
8,1.273800,1.221188
10,1.241300,1.206817
12,1.239000,1.194841
14,1.199300,1.186904
16,1.221000,1.178792
18,1.257800,1.170574
20,1.177800,1.163446


wandb: Adding directory to artifact (./MistralAI_iwslt15_en_vi_manual/checkpoint-31)... Done. 1.1s


In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

1152.6625 seconds used for training.
19.21 minutes used for training.
Peak reserved memory = 8.693 GB.
Peak reserved memory for training = 4.068 GB.
Peak reserved memory % of max memory = 58.944 %.
Peak reserved memory for training % of max memory = 27.583 %.


In [ ]:
trainer_stats

TrainOutput(global_step=31, training_loss=1.2762694435734903, metrics={'train_runtime': 1152.6625, 'train_samples_per_second': 0.868, 'train_steps_per_second': 0.027, 'total_flos': 9001980202844160.0, 'train_loss': 1.2762694435734903, 'epoch': 0.99})